In [ ]:
from operator import itemgetter

from langchain.prompts import ChatPromptTemplate
from langchain.embeddings import LlamaCppEmbeddings
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.vectorstores import FAISS
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [ ]:
n_gpu_layers = 1  # Metal set to 1 is enough.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
# llama_model_path = "../../models/zephyr-7b-beta.Q4_K_M.gguf"
llama_model_path = "../../models/zephyr-7b-beta.Q8_0.gguf"

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path=llama_model_path,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx=2048,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    verbose=True,
)

In [ ]:
#Use Llama model for embedding
embeddings = LlamaCppEmbeddings(model_path=llama_model_path, n_ctx=2048)

In [ ]:
output = llm("Q: Name the planets in the solar system? A: ", max_tokens=32, stop=["Q:", "\n"], echo=True)

In [ ]:
output

In [ ]:
vectorstore = FAISS.from_texts(["harrison worked at kensho"], embedding=embeddings)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = llm

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | prompt 
    | model 
    | StrOutputParser()
)

In [ ]:
chain.invoke("where did harrison work?")


In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = {
    "context": itemgetter("question") | retriever, 
    "question": itemgetter("question"), 
    "language": itemgetter("language")
} | prompt | model | StrOutputParser()

In [ ]:
chain.invoke({"question": "where did harrison work", "language": "italian"})

In [ ]:
llm("Simulate a rap battle between Stephen Colbert and John Oliver")